In [152]:
### Grading script code 
### You don't need to read this, proceed to the next cell
import sys
import functools
ipython = get_ipython()

def set_traceback(val):
    method_name = "showtraceback"
    setattr(
        ipython,
        method_name,
        functools.partial(
            getattr(ipython, method_name),
            exception_only=(not val)
        )
    )

class AnswerError(Exception):
  def __init__(self, message):
    pass

def exec_test(f, question):
    try:
        f()
        print(question + " Pass")
    except:
        set_traceback(False) # do not remove
        raise AnswerError(question + " Fail")

# Week 5 Problem Set

## Homework

**HW1.** *Dictionary:* Write two functions:
1. `count_degrees(G)`: which sums up the degrees of all vertices in the graph. The degree of a vertex is defined as the number of edges connected to a Vertex. The argument `G` is a dictionary that represents the graph.
2. `count_edges(G)`: which counts the number of edges in the graph. An edge is defined as a connection between two vertices. The argument `G` is a dictionary.

In [153]:
def count_degrees(G):
    return sum([len(ls) for ls in G.values()])

def count_edges(G):
    # generate edge-pairs and remove duplicates to account for double-edges e.g A to B and B to A, then get length of set
    return len(set([tuple_sort((key, value_of_ls)) for key, ls in G.items() for value_of_ls in ls]))

def tuple_sort(tup):
    return tup if tup[1] > tup[0] else tup[::-1]


In [154]:
g1 = {"A": ["B", "E"],
      "B": ["A", "C"],
      "C": ["B", "D", "E"],
      "D": ["C"],
      "E": ["A", "C"]}

d = count_degrees(g1)
e = count_edges(g1)

assert d == 10
assert e == 5

In [155]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Create a class called `GraphSearch` which is a subclass of the class `Graph`. This class should override the method `_create_vertex(id)` and instantiate a `VertexSearch` object instead of `Vertex`.

In [156]:
class Vertex:
    def __init__(self, id_=""):
        self.id_ = id_
        self.neighbours = {}
    
    def add_neighbour(self, nbr_vertex, weight=0):
        self.neighbours[nbr_vertex] = weight
    
    def get_neighbours(self):
        return [neighbour for neighbour in self.neighbours.keys()]
    
    def get_weight(self, neighbour):
        return self.neighbours.get(neighbour)
    
    def __eq__(self, other):
        return self.id_ == other.id_
    
    def __lt__(self, other):
        return self.id_ < other.id_

    def __hash__(self):
        return hash(self.id_)
    
    def __str__(self):
        return f"Vertex {self.id_} is connected to: {', '.join([neighbour.id_ for neighbour in self.neighbours.keys()])}"

In [157]:
import sys

class VertexSearch(Vertex):
    # calling __init__ means that it does not inherit __init__ from Vertex
    def __init__(self, *args):
        # VertexSearch needs to inherit Vertex's __init__ and add stuff
        Vertex.__init__(self, *args)
        self.colour = "white"
        self.d = sys.maxsize
        self.f = sys.maxsize
        self.parent = None


In [158]:
class Graph:
    def __init__(self):
        self.vertices = {}
        
    def _create_vertex(self, id_):
        return Vertex(id_)
    
    def add_vertex(self, id_):
        self.vertices[id_] = self._create_vertex(id_)
    
    def get_vertex(self, id_):
        return self.vertices.get(id_)
    
    def add_edge(self, start_v, end_v, weight=0):
       self.vertices[start_v].add_neighbour(self.vertices[end_v], weight)
        
    def get_neighbours(self, id_):
        return [neighbour.id_ for neighbour in self.vertices[id_].get_neighbours()]
    
    def __contains__(self, id_):
        return id_ in self.vertices.keys()
    
    def __iter__(self):
        for k,v in self.vertices.items():
            yield v 
    
    @property
    def num_vertices(self):
        return len(self.vertices)

In [159]:
class GraphSearch(Graph):
    def _create_vertex(self, id_):
        return VertexSearch(id_)


In [160]:
g2 = GraphSearch()
g2.add_vertex("Z")
assert(type(g2.vertices["Z"]) == type(VertexSearch()))

In [161]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Undirected Graph:* **You need to complete CS5 and CS6 before you do this homework.** Paste your answer for `Search2D` and `SearchBFS` classes into the cell below.

Create a subclass of `GraphSearch` class called `UGraphSearch` for undirected graphs. All edges in undirected graphs are bidirectional (i.e. vertex1 <-> vertex2). 
This means that you need to override the `add_edge()` method.

In [162]:
class Queue:
    def __init__(self):
        self.__items = []
    
    def enqueue(self, item):
        self.__items.append(item)
    
    def dequeue(self):
        return self.__items.pop(0)
    
    def peek(self):
        return self.__items[0]
    
    @property
    def is_empty(self):
        return self.__items == []

    @property
    def size(self):
        return len(self.__items)

In [163]:
import sys

class Search2D:
    def __init__(self, g):
        self.graph = g
    
    def clear_vertices(self):
        # for each vertice that is contained in self.graph, result its VertexSearch values
        for vertice in self.graph.vertices.values():
            vertice.colour = "white"
            vertice.d = sys.maxsize
            vertice.f = sys.maxsize
            vertice.parent = None
    
    def __iter__(self):
        return iter([v for v in self.graph])
    
    def __len__(self):
        return len([v for v in self.graph.vertices])

In [164]:
class SearchBFS(Search2D):
    def search_from(self, start):
        vertex_queue = Queue()

        starting_vertice = self.graph.vertices.get(start)
        # added a if condition for invalid starts
        if not starting_vertice:
            return None
        # start from starting vertice, set colour to "grey" since we haven't touched the neighbour 
        starting_vertice.colour = "grey"
        # initialize d to 0 since this is our start
        starting_vertice.d = 0
        # enqueue the starting node
        vertex_queue.enqueue(starting_vertice)
        # while queue is not empty: 
        while vertex_queue.is_empty != True:
            # obtain vertex in queue
            current_vertex = vertex_queue.dequeue()

            # for each neighbour of the current vertex: 
            for neighbour in current_vertex.get_neighbours():
                # if we have not explored this neighbour, its colour should be 'white'
                if neighbour.colour == "white":
                    neighbour.colour = "grey"
                    # its distance shall adopt the parent's + 1
                    neighbour.d = current_vertex.d + 1
                    # set the neighbour's parent to current_vertex
                    neighbour.parent = current_vertex
                    # enqueue neighbour
                    vertex_queue.enqueue(neighbour)

            # we have explored current_vertex, so set 'black'
            current_vertex.colour = "black"
    
    def get_shortest_path(self, start, dest):
        # if either start or dest exists, return 
        if not (self.graph.vertices.get(start) and self.graph.vertices.get(dest)):
            return
            
        # if start == dest, no need to do anything just return [start]
        if start == dest:
            return [start]
        # clear everything so we can start search and clear nodes
        self.clear_vertices()
        # explore from start
        self.search_from(start)
        # return recursive get_path call
        return self.get_path(self.graph.vertices.get(start), self.graph.vertices.get(dest).parent, [dest])
    
    def get_path(self, start, dest, result):
        if result[-1] == start.id_:
            # result is flipped because we started from the child, where we really want to start from oldest parent
            return result[::-1]
        elif dest:
            # if dest is not None, append parent id to result and do recursive call
            result.append(dest.id_)
            return self.get_path(start, dest.parent, result)
        else: 
            # means that dest is None, no paths that lead from dest to start
            return ["No Path"]

In [165]:
class UGraphSearch(GraphSearch):
    def add_edge(self, start_v, end_v, weight=0):
        # going from starting vertice to ending vertice is now bidirectional, i.e ending vertice to starting vertice is also added
        ending_vertice = self.vertices.get(end_v)
        start_vertice = self.vertices.get(start_v)

        # recall that key-value pairs in Vertex.neighbours is {Vertex: weight}
        start_vertice.add_neighbour(ending_vertice, weight)
        ending_vertice.add_neighbour(start_vertice, weight)

In [166]:
g2 = UGraphSearch()
assert g2.vertices == {} and g2.num_vertices == 0
g2.add_vertex("L")
g2.add_vertex("I")
g2.add_vertex("S")
g2.add_vertex("P")
assert g2.num_vertices == 4
assert "L" in g2
assert "I" in g2
assert "S" in g2
assert "P" in g2
g2.add_edge("L", "I")
g2.add_edge("I", "S")
g2.add_edge("S", "P")
assert sorted(g2.get_neighbours("L")) == ["I"]
assert sorted(g2.get_neighbours("I")) == ["L", "S"]
assert sorted(g2.get_neighbours("S")) == ["I", "P"]
assert sorted(g2.get_neighbours("P")) == ["S"]

In [167]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [168]:
g2 = UGraphSearch()
g2.add_vertex("r")
g2.add_vertex("s")
g2.add_vertex("t")
g2.add_vertex("u")
g2.add_vertex("v")
g2.add_vertex("w")
g2.add_vertex("x")
g2.add_vertex("y")
g2.add_vertex("z")
g2.add_edge("r", "s")
g2.add_edge("r", "v")
g2.add_edge("s", "w")
g2.add_edge("t", "u")
g2.add_edge("t", "x")
g2.add_edge("t", "w")
g2.add_edge("u", "t")
g2.add_edge("u", "x")
g2.add_edge("u", "y")
g2.add_edge("v", "r")
g2.add_edge("w", "s")
g2.add_edge("w", "t")
g2.add_edge("w", "x")
g2.add_edge("x", "w")
g2.add_edge("x", "t")
g2.add_edge("x", "u")
g2.add_edge("x", "y")
g2.add_edge("y", "u")
g2.add_edge("y", "x")
gs = SearchBFS(g2)
gs.search_from("s")
assert gs.graph.get_vertex("s").d == 0
assert gs.graph.get_vertex("t").d == 2
assert gs.graph.get_vertex("u").d == 3
assert gs.graph.get_vertex("v").d == 2
assert gs.graph.get_vertex("w").d == 1
assert gs.graph.get_vertex("x").d == 2
assert gs.graph.get_vertex("y").d == 3
assert gs.graph.get_vertex("r").d == 1
ans = gs.get_shortest_path("s", "u")
assert ans == ["s", "w", "t", "u"] or ans == ["s", "w", "x", "u"]
ans = gs.get_shortest_path("v", "s")
assert ans == ["v", "r", "s"]
ans = gs.get_shortest_path("v", "y")
assert ans == ["v", "r", "s", "w", "x", "y"]

In [169]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Depth-First-Search:* Create a class `SearchDFS` as a child class of `Search2D` to implement Depth-First-Search algorithm. You should add one additional attribute:
- `time`: which is an attribute to record the discovery time and the finishing time.

The class should also implement the following methods:
- `search()`: which modifies the vertices' properties such as `colour`, `d`, and `parent` following Depth-First-Search algorithm.
- `dfs_visit(vert)`: which is the recursive method for visiting a vertex in Depth-First-Search algorithm.

In [170]:
import sys

class SearchDFS(Search2D):
    def __init__(self, g):
        self.graph = g
        # time is really just the number of nodes explored
        self.time = 0
      
    def search(self):
        for vertice in self.graph.vertices.values():
            # if vertice has not been visited, do dfs_visit
            if vertice.colour == "white":
                self.dfs_visit(vertice)

    
    def dfs_visit(self, vert):
        # every time dfs_visit is called, a node is being explored, so self.time += 1
        self.time += 1
        vert.d = self.time
        vert.colour = "grey"
        for neighbour in vert.get_neighbours():
            if neighbour.colour == "white":
                neighbour.parent = vert
                self.dfs_visit(neighbour)

        # at this step, the node is being 'revisited', so we set the node to black and self.time += 1
        vert.colour =  "black"
        self.time += 1

        # then we set vert.f, i.e the finishing time to self.time
        vert.f = self.time


In [171]:
g4 = GraphSearch()
g4.add_vertex("e")
g4.add_vertex("m")
g4.add_vertex("a")
g4.add_vertex("c")
g4.add_vertex("s")
g4.add_edge("e", "m")
g4.add_edge("m", "a")
g4.add_edge("a", "c")
g4.add_edge("c", "s")
gs = SearchDFS(g4)
gs.search()
assert gs.graph.get_vertex("e").parent == None 
assert gs.graph.get_vertex("m").parent == gs.graph.get_vertex("e")
assert gs.graph.get_vertex("m").d == 2 and gs.graph.get_vertex("a").f == 8
assert gs.graph.get_vertex("c").d == 4 and gs.graph.get_vertex("s").f == 6

In [172]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW5.** *Topological Sort:* Create a function that takes in a `SearchDFS` object to perform a topological sort:
- `topological_sort(g)`: which takes in a `SearchDFS` object and returns a list of `VertexSearch` objects sorted based on their `f` attribute. This method should call the `search()` method of the `SearchDFS` to first calculate the `f` attribute of the vertices. Note that you need to copy the `SearchDFS` object of your input argument so as not to mutate the object.

In [173]:
import sys
import copy

def topological_sort(g):
    # replace pointer variable g to a copy of SearchDFS behind g (confusing i know)
    g = copy.copy(g)
    g.search()
    return sorted(g.graph.vertices.values(), key=lambda x: x.f, reverse=True)



In [174]:
import copy
g = GraphSearch()
g.add_vertex("3/4 cup milk")
g.add_vertex("1 egg") 
g.add_vertex("1 tbl oil")
g.add_vertex("1 cup mix")
g.add_vertex("heat syrup")
g.add_vertex("heat griddle")
g.add_vertex("pour 1/4 cup")
g.add_vertex("turn when bubbly")
g.add_vertex("eat")
g.add_edge("3/4 cup milk", "1 cup mix")
g.add_edge("1 egg", "1 cup mix")
g.add_edge("1 tbl oil", "1 cup mix")
g.add_edge("1 cup mix", "heat syrup")
g.add_edge("1 cup mix", "pour 1/4 cup")
g.add_edge("heat griddle", "pour 1/4 cup")
g.add_edge("pour 1/4 cup", "turn when bubbly")
g.add_edge("turn when bubbly", "eat")
g.add_edge("heat syrup", "eat")
gs = SearchDFS(g)  

path = topological_sort(gs)
ans = [v.f for v in copy.copy(path)]
assert ans == [18, 16, 14, 12, 11, 10, 9, 6, 5]
ans = [v.id_ for v in copy.copy(path)]
assert ans == ['heat griddle', '1 tbl oil', '1 egg', '3/4 cup milk', '1 cup mix', 'pour 1/4 cup', 'turn when bubbly', 'heat syrup', 'eat']

In [175]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
